### Parallel Computing

## Part 3 of 5
# What to parallelize

## Reminder
<a href="#/slide-2-0" class="navigate-right" style="background-color:blue;color:white;padding:8px;margin:2px;font-weight:bold;">Continue with the lesson</a>

<br>
</br>
<font size="+1">

By continuing with this lesson you are granting your permission to take part in this research study for the Hour of Cyberinfrastructure: Developing Cyber Literacy for GIScience project. In this study, you will be learning about cyberinfrastructure and related concepts using a web-based platform that will take approximately one hour per lesson. Participation in this study is voluntary.

Participants in this research must be 18 years or older. If you are under the age of 18 then please exit this webpage or navigate to another website such as the Hour of Code at https://hourofcode.com, which is designed for K-12 students.

If you are not interested in participating please exit the browser or navigate to this website: http://www.umn.edu. Your participation is voluntary and you are free to stop the lesson at any time.

For the full description please navigate to this website: <a href="../../gateway-lesson/gateway/gateway-1.ipynb">Gateway Lesson Research Study Permission</a>.

</font>

In [ ]:
# This code cell starts the necessary setup for Hour of CI lesson notebooks.
# First, it enables users to hide and unhide code by producing a 'Toggle raw code' button below.
# Second, it imports the hourofci package, which is necessary for lessons and interactive Jupyter Widgets.
# Third, it helps hide/control other aspects of Jupyter Notebooks to improve the user experience
# This is an initialization cell
# It is not displayed because the Slide Type is 'Skip'

from IPython.display import HTML, IFrame, Javascript, display
from ipywidgets import interactive
import ipywidgets as widgets
from ipywidgets import Layout, Textarea
import warnings
import getpass # This library allows us to get the username (User agent string)

# import package for hourofci project
import sys
sys.path.append('../../supplementary') # relative path (may change depending on the location of the lesson notebook)
import hourofci


# load javascript to initialize/hide cells, get user agent string, and hide output indicator
# hide code by introducing a toggle button "Toggle raw code"
HTML(''' 
    <script type="text/javascript" src=\"../../supplementary/js/custom.js\"></script>
    
    <style>
        .output_prompt{opacity:0;}
    </style>
    
    <input id="toggle_code" type="button" value="Toggle raw code">
''')

## Data parallelism
Notice when we were talking about Sam, Parker, and Patricia that the three workers dividing the work and all doing the same task: planting seeds in a single field. This is an example of **data parallelism**, which involves dividing data into chunks and distributing the chunks to a number of workers to work on in parallel. In this case we have 3 workers who divide the field into partitions and then plant seeds serially (Sam) or in parallel (Parker and Patricia).

## Data parallelism
Here you can see a dataset (left) and then we can divide it into the top half (middle) and bottom half (right). If we assign the top half to one worker and the bottom half to another worker. We could speed up our data processing by up to 2X. This is because each worker will do half of the work, and the work will be done simultaneously.

<center><img src='supplementary/blocks.png' width=500></center>

## Another type of parallelism: Task parallelism
Are there other types of parallelism? Yes! Another common type of parallelism is called task parallelism. In task parallelism different workers divide the problem by completing different tasks oftentimes in parallel. In our field problem if we need to water the seeds after they are planted, then we can invoke task parallelism. We can assign one worker to water each seed after it is planted. 

## We see these types of parallelism all over.
When you shop for groceries and you go to check out, you might notice a row of checkout lanes. Each lane has a worker to check you and your fellow shoppers out. This is similar to data parallelism. Taking the same problem (checking out a large number of shoppers) and dividing the work among a number of workers (people working each checkout lane). Before you finished shopping, you might have met someone behind the meat counter cutting fresh ham, another worker stocking shelves, and yet another mopping the floors. This is an example of task parallelism where each worker is performing a different task, which are all necessary for you and a large number of shoppers to complete your respective shopping trips.


## We see these types of parallelism all over.
Returning to our cooking example, we invoke task parallelism when we ask our friend to cut vegetables while we are putting a casserole in the oven. If there are a lot of vegetables to cut and we join our friend, then we are invoking the equivalent of data parallelism where we are dividing the work to finish it faster.


## Task versus Data Parallelism
On the right we have Data Parallelism: Processing Core 0 completes Task A for the top half of the data, then completes Task B for the top of the data. Processing Core 1 Task A for the bottom half of the data, then completes Task B for the bottom of the data. Both are completed independently (but in parallel).
<center><img src='supplementary/par_types.png' width=700></center>

Notice in this example the processing time is theoretically equivalent. 

## How could we speed up the processing even further?
<center><img src='supplementary/par_types.png' width=500></center>

If we increase the number of processing cores to 4, then …

In [ ]:
tarea1 = Textarea(
            value='',
            placeholder='Type your answer here',
            description='',
            disabled=False,
            layout=Layout( height='100px', min_height='100px', width='500px')
            )



def out():
    return print('Submitted!')
display(tarea1)

hourofci.SubmitBtn2(tarea1, out)

## How could we speed up the processing even further?
Right, we could apply BOTH data parallelism and task parallelism. Assigning one task (A or B) and one half of the data (top or bottom) to each core. Like so:

<center><img src='supplementary/par_types2.png' width=700></center>

Great work!

## Notice
Notice in Data Parallelism that we are taking the data and decomposing it into multiple portions. In our examples we decomposed the data into the top half and the bottom half. What if the data were spatial? Then dividing the top half and the bottom half would be like dividing the United States into the Northern half and the Southern half. Are there other ways we could divide these types of data?

## Did you know?
That there is a term for decomposing spatial data, it is called **Spatial Domain Decomposition**. There are numerous ways to decompose spatial data. Here are a few common examples. Each have their own advantages and disadvantages in their complexity and flexibility.

<center><img src='supplementary/grid.png' width=700></center>


## Inter-processor communication
Parallelism does not come for free. Amdahl's Law shows us that the 'serial' portion of a code will limit our speedup. One important aspect of parallelism is coordinating all of the processing across all of the cores. This happens using Inter-processor Communication. For example, if we divide the US into the Northern half and Southern half. We may need information from one half to be made available for the other half. So we need a way for processing cores to communicate with each other.

<center><img src='supplementary/usa.png' width=600></center>


## A Tale of Two… Paradigms for Communication

<center><img src='supplementary/tale.png' width=700></center>

<b>Shared Memory</b> is one paradigm for inter-processor communication. In this paradigm, there is a single shared memory space that processors can write to and read from. If they use the same memory location (storing a Latitude and Longitude coordinate pair, for example), then they can communicate through the shared memory.<br>

## Shared Memory

<table>
    <tr style="background: #fff; text-align: left; vertical-align:">
        <td style="background: #fff; text-align: left; font-size: 23px;">
An analogy for shared memory is a shared sheet of paper. If we give two people a sheet of paper, where they can each write questions and answers. They will be able to easily communicate without talking. What might be difficult is if they both wanted to write something at the same time. Or if one person wrote over the writing of the other person. Or if they both wrote a question and were waiting on a response from the other person. These are all examples of <b>resource contention</b>. Here we can have conflicts in accessing a shared resource (the sheet of paper). 
        </td>
        <td style="width: 30%; background: #fff; text-align: left; vertical-align: top;"> 
            <img src="supplementary/shared_mem.png" width="100%">
    </td>
    </tr>
</table>

## A Tale of Two… Paradigms for Communication

<center><img src='supplementary/tale.png' width=700></center>

<b>Message Passing</b> is another popular paradigm. In this paradigm, each processing core has its own memory space. So, the Latitude and Longitude coordinate pair on Processing Core 0 is not accessible to Processing Core 1. So, Processing Core 0 must send a message with the data inside the message. Processing Core 1 accepts the message and now has the data.

## Message Passing

<table>
    <tr style="background: #fff; text-align: left; vertical-align:">
        <td style="background: #fff; text-align: left; font-size: 23px;">
            An analogy for message passing is sending notes using the postal system. Let's give two people unlimited sheets of papers, envelopes, and stamps. They can each write questions and answers and post them to the other person. They will be able to easily communicate without talking. What might be difficult in this scenario is sending messages simultaneously because they do not know when the other person is sending them a post! <br>
            There is no possibility that one person overwrites the note of another, because they each write their notes separately and send them. However, what if they both wrote a question and were waiting on a response from the other person? What if they write so many notes that they overwhelm the other person? Or what if so many notes are going through the postal system that the postal system is overwhelmed? The last example is called <b>network contention</b>. Here we can have conflicts in sending and receiving our notes. 
        </td>
        <td style="width: 30%; background: #fff; text-align: left; vertical-align: top;"> 
            <img src="supplementary/mess_passing.png" width="100%">
    </td>
    </tr>
</table>

## Let's check.
Let's test your knowledge. We have two parallel processes running. Process 0 writes the value 0 into the first element a dataset like so: 

`dataset[0] = 0`

Process 1 reads the value written by Process and assigns it to a variable like so:

`variable = dataset[0]`
Is this an example of shared memory or message passing?


In [ ]:
o1='Shared Memory'
o2='Message Passing'
widget12 = widgets.RadioButtons(
    options = [o1, o2],
    description = '', style={'description_width': 'initial'},
    layout = Layout(width='100%'),
    value = None)

display(widget12)
hourofci.SubmitBtn2(widget12)


## Let's check: Shared memory!
It is an example of shared memory. Here we are using the fact that the `dataset` is accessible to both processes. So each process can write to (i.e., `dataset[0] = 0`) and read from (i.e., `variable = dataset[0]`) the same shared data.





## Speedup
Now that we understand Amdahl's Law, Task versus Data Parallelism, and Inter-processor Communication we can understand why we measure **Speedup**.

What is speedup?


## Speedup
Speedup is commonly used to assess the performance of a parallel program.  Speedup is defined as the execution time on a single core (T<sub>1</sub>) over the execution time on p cores (T<sub>p</sub>) (Amdahl, 1967). Linear or ideal speedup is reached when S<sub>p</sub> = p. 

<center><img src="supplementary/speedup.png" width="700"></center>


## Speedup
<center><img src="supplementary/speedup.png" width="700"></center>
Every time we introduce serial portions of our code and/or inter-processor communication and/or resource contention, we reduce our actual speedup from our theoretically ideal (a.k.a. linear) speedup.



## Let's see! Back to the field

<center><img src="supplementary/portion.png" width="700"></center>
On the left we have 5 ordered tasks. 3 of the tasks are completed sequentially. Two tasks are completed in parallel. If we use our three farmers: Sam, Parker, and Patricia, then we can see that only 1 portion of this problem can be parallelized. 

## Let's check

<center><img src="supplementary/portion.png" width="700"></center>
Using this diagram, what will be the key limitation in achieving ideal speedup?

In [ ]:
tarea2 = Textarea(
            value='',
            placeholder='Type your answer here',
            description='',
            disabled=False,
            layout=Layout( height='100px', min_height='100px', width='500px')
            )



def out2():
    return print('Answer submitted! Go to next page to check your answer.')
display(tarea2)

hourofci.SubmitBtn2(tarea2, out2)

## Let's check! The serial portions!
The serial portion will limit the amount of speedup that we can achieve. Recall Amdahl's Law.
If we assume *P* is the parallel portion of a parallel program, 
then *(1-P)* is the portion that cannot be made parallel (serial portion).
Amdahl's law states that the maximum speedup on *N* processors is:

$$
  S(N) = \frac{1}{(1-P)+\frac{P}{N}}
$$

Here, Sam's serial portion *(1-P)* is quite large. So this will limit our speedup. Our goal is to limit Sam's portion so that we can parallelize more of our problem.


## Next up
Now, enough talking. Let's parallelize some code!
In the next Exploration Segment you will start with a simple piece of code and then walk through a step-by-step process introducing parallel computing so that we can take these principles and put them to action.






<font size="+1"><a style="background-color:blue;color:white;padding:12px;margin:10px;font-weight:bold;" 
href="pc-exploration.ipynb">Click here to go to the next notebook.</a></font>
